# version 1

In [1]:
import json
import os

def convert_to_ufo(txt_path, img_w, img_h):
    # 파일명 추출 (확장자 제외)
    image_filename = os.path.basename(txt_path).replace('.txt', '.jpg')
    
    # UFO 형식의 기본 구조 생성
    ufo_data = {
        "images": {
            image_filename: {
                "paragraphs": {},
                "words": {},
                "chars": {},
                "img_w": img_w,
                "img_h": img_h,
                "num_patches": None,
                "tags": [],
                "relations": {},
                "annotation_log": {
                    "worker": "worker",
                    "timestamp": "2024-03-22",
                    "tool_version": "",
                    "source": None
                },
                "license_tag": {
                    "usability": True,
                    "public": False,
                    "commercial": True,
                    "type": None,
                    "holder": "Upstage"
                }
            }
        }
    }
    
    # txt 파일 읽기
    with open(txt_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    # 각 라인을 처리
    for idx, line in enumerate(lines, start=1):
        # 쉼표로 분리
        parts = line.strip().split(',')
        if len(parts) == 9:  # x1,y1,x2,y2,x3,y3,x4,y4,text 형식인지 확인
            x1, y1, x2, y2, x3, y3, x4, y4, text = parts
            
            # word 정보 추가
            word_id = f"{idx:04d}"  # 4자리 숫자 형식 (0001, 0002, ...)
            ufo_data["images"][image_filename]["words"][word_id] = {
                "transcription": text,
                "points": [
                    [float(x1), float(y1)],
                    [float(x2), float(y2)],
                    [float(x3), float(y3)],
                    [float(x4), float(y4)]
                ]
            }
    
    return ufo_data

def save_ufo_file(ufo_data, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(ufo_data, f, ensure_ascii=False, indent=2)

# 사용 예시
txt_path = "/data/ephemeral/home/Dong_Yeong/level2-cv-datacentric-cv-06/code/data/storie/X51009453804.txt"
output_path = "tX51009453804.json"
img_w = 1013  # 이미지 실제 너비로 변경 필요
img_h = 463  # 이미지 실제 높이로 변경 필요

ufo_data = convert_to_ufo(txt_path, img_w, img_h)
save_ufo_file(ufo_data, output_path)

# Version 2

In [2]:
import json
import os
from PIL import Image
import re

def get_image_size(image_path):
    """이미지 파일의 크기를 반환하는 함수"""
    try:
        with Image.open(image_path) as img:
            return img.size
    except Exception as e:
        print(f"Warning: Could not get image size for {image_path}: {e}")
        return None, None

def get_base_image_name(filename):
    """파일명에서 (1), (2) 등을 제거하고 기본 이미지 이름을 반환"""
    # (1), (2) 등의 패턴을 제거
    base_name = re.sub(r'\(\d+\)', '', filename)
    # .txt 확장자 제거하고 .jpg 확장자 추가
    return base_name.replace('.txt', '.jpg')

def convert_to_ufo(data_dir):
    ufo_data = {"images": {}}
    
    # txt 파일 목록 가져오기
    txt_files = [f for f in os.listdir(data_dir) if f.endswith('.txt')]
    
    # 각 txt 파일 처리
    for txt_file in txt_files:
        txt_path = os.path.join(data_dir, txt_file)
        
        # 기본 이미지 이름 구하기
        image_filename = get_base_image_name(txt_file)
        image_path = os.path.join(data_dir, image_filename)
        
        # 이미지 크기 가져오기
        img_w, img_h = get_image_size(image_path)
        if img_w is None or img_h is None:
            print(f"Skipping {txt_file} due to missing image")
            continue
            
        # 새로운 이미지 엔트리 생성
        if image_filename not in ufo_data["images"]:
            ufo_data["images"][image_filename] = {
                "paragraphs": {},
                "words": {},
                "chars": {},
                "img_w": img_w,
                "img_h": img_h,
                "num_patches": None,
                "tags": [],
                "relations": {},
                "annotation_log": {
                    "worker": "worker",
                    "timestamp": "2024-03-22",
                    "tool_version": "",
                    "source": None
                },
                "license_tag": {
                    "usability": True,
                    "public": False,
                    "commercial": True,
                    "type": None,
                    "holder": "Upstage"
                }
            }
        
        # txt 파일 읽기
        with open(txt_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        # 현재 이미지의 word 수 계산
        current_word_count = len(ufo_data["images"][image_filename]["words"])
        
        # 각 라인 처리
        for idx, line in enumerate(lines, start=1):
            parts = line.strip().split(',')
            if len(parts) == 9:
                x1, y1, x2, y2, x3, y3, x4, y4, text = parts
                
                # word ID 생성 (기존 word 수에 현재 인덱스를 더함)
                word_id = f"{current_word_count + idx:04d}"
                
                ufo_data["images"][image_filename]["words"][word_id] = {
                    "transcription": text,
                    "points": [
                        [float(x1), float(y1)],
                        [float(x2), float(y2)],
                        [float(x3), float(y3)],
                        [float(x4), float(y4)]
                    ]
                }
    
    return ufo_data

def save_ufo_file(ufo_data, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(ufo_data, f, ensure_ascii=False, indent=2)

# 실행 코드
data_dir = "/data/ephemeral/home/Dong_Yeong/level2-cv-datacentric-cv-06/code/data/storie"
output_path = os.path.join(data_dir, "train.json")

ufo_data = convert_to_ufo(data_dir)
save_ufo_file(ufo_data, output_path)
print("UFO 파일 생성 완료!")

UFO 파일 생성 완료!
